In [1]:
from SNEWS_PT.hop_pub import Publisher_Coincidence_Tier
from datetime import datetime
from datetime import datetime

def nu_t():
    return datetime.utcnow().strftime("%H:%M:%S:%f")

my_detector = 'DS-20K'

pub = Publisher_Coincidence_Tier(my_detector)

pub.send_coincidence_tier_message(nu_time=nu_t(), p_value =  0.67)

---------------------------------------------------------
_id                 :18_CoincidenceTier_21/11/12_22:05:39:244113
detector_name       :DS-20K
sent_time           :21/11/12 22:05:39
machine_time        :None
neutrino_time       :22:05:39:244092
p_value             :0.67


In [2]:
from SNEWS_PT.hop_pub import Publisher_Significance_Tier
from datetime import datetime


def nu_t():
    return datetime.utcnow().strftime("%H:%M:%S:%f")

my_detector = 'DS-20K'

pub = Publisher_Significance_Tier(my_detector)

pub.send_sig_tier_message(nu_time=nu_t(), p_values =  [0.67,0.87,0.60])

---------------------------------------------------------
_id                 :18_SigTier_21/11/12_22:05:40:592718
detector_name       :DS-20K
sent_time           :21/11/12 22:05:40
machine_time        :None
neutrino_time       :22:05:40:592679
p_values            :[0.67, 0.87, 0.6]


In [3]:
from SNEWS_PT.hop_pub import Publisher_Timing_Tier
from datetime import datetime

def nu_t():
    return datetime.utcnow().strftime("%H:%M:%S:%f")

my_detector = 'DS-20K'

pub = Publisher_Timing_Tier(my_detector)

pub.send_t_tier(nu_time=nu_t(), timing_series = ['16:23:24"4546','16:23:24"4666','16:23:24"4756'])

---------------------------------------------------------
_id                 :18_TimeTier_21/11/12_22:05:41:927918
detector_name       :DS-20K
sent_time           :21/11/12 22:05:41
machine_time        :None
neutrino_time       :22:05:41:927809
timing_series       :['16:23:24"4546', '16:23:24"4666', '16:23:24"4756']


In [4]:
from SNEWS_PT.hop_pub import Publisher_Rectraction
from datetime import datetime


my_detector = 'DS-20K'

pub = Publisher_Rectraction(my_detector)

pub.retract(which_tier='CoincidenceTier', n_retract_latest=5, retraction_reason='someone spilled coffee on the DAQ')

---------------------------------------------------------
IT'S OKAY, WE ALL MAKE MISTAKES
_id                 :18_FalseOBS_21/11/12_22:05:44:286197
detector_name       :DS-20K
sent_time           :21/11/12 22:05:44
machine_time        :None
false_id            :None
which_tier          :CoincidenceTier
N_retract_latest    :5
retraction_reason   :someone spilled coffee on the DAQ
